In [6]:
import os
os.getcwd()

'C:\\Users\\baron\\OneDrive\\Documents\\cse6250group\\notebooks'

In [7]:
#read and pivot data into appropriate shape
path = 'C:/Users/baron/OneDrive/Documents/cse6250group/data/'
import pandas as pd

data = pd.read_csv(path + 'shhs1-hrv-5min-0.15.0.csv')
df = data.melt(id_vars = ['nsrrid','Start__sec_'], var_name = '30_sec_interval', value_name = 'sleep_stage')
df ['sleep_stage'] = df['sleep_stage'].replace({0: 0, 1: 1, 2: 2, 3: 3, 4: 3, 5: 5, 9: 0})
df['30_sec_interval'] = df['30_sec_interval'].astype(int)
df['epoch'] = df['Start__sec_'] + df['30_sec_interval'].astype(int)
df['session'] = 1


data2 = pd.read_csv(path + 'shhs2-hrv-5min-0.15.0.csv')
df2 = data2.melt(id_vars = ['nsrrid','Start__sec_'], var_name = '30_sec_interval', value_name = 'sleep_stage')
df2['sleep_stage'] = df2['sleep_stage'].replace({0: 0, 1: 1, 2: 2, 3: 3, 4: 3, 5: 5, 9: 0})
df2['30_sec_interval'] = df2['30_sec_interval'].astype(int)
df2['epoch'] = df2['Start__sec_'] + df2['30_sec_interval'].astype(int)
df2['session'] = 2


final = pd.concat([df, df2])

final.to_csv(path + 'cleaned_sleep_stage_data.csv')





In [8]:
#group data into sleep instances
import pandas as pd 
sleep_stage_data = pd.read_csv(path + 'cleaned_sleep_stage_data.csv')
sleep_stage_data = sleep_stage_data[['nsrrid','session','epoch','sleep_stage']]
sleep_stage_data = sleep_stage_data.groupby(['nsrrid','session','sleep_stage']).count()
sleep_stage_data = sleep_stage_data.reset_index()
sleep_pivot = sleep_stage_data.pivot_table(index=['nsrrid','session'], columns='sleep_stage', values='epoch')
sleep_pivot = sleep_pivot.reset_index()


sleep_pivot['total_stage'] = sleep_pivot[0] + sleep_pivot[1] + sleep_pivot[2] + sleep_pivot[3] + sleep_pivot[5]
sleep_pivot['w%'] = sleep_pivot[0]/sleep_pivot['total_stage']*100
sleep_pivot['n1%'] = sleep_pivot[1]/sleep_pivot['total_stage']*100
sleep_pivot['n2%'] = sleep_pivot[2]/sleep_pivot['total_stage']*100
sleep_pivot['n3%'] = sleep_pivot[3]/sleep_pivot['total_stage']*100
sleep_pivot['rem%'] = sleep_pivot[5]/sleep_pivot['total_stage']*100






In [10]:
#join incremental patient data

supplement1 = pd.DataFrame(pd.read_csv(path + 'shhs1-dataset-0.15.0.csv', encoding='cp1252'))
supplement1 = supplement1.rename(columns=str.lower)
supplement1 = supplement1[['nsrrid', 'age_s1', 'gender', 'race','bp_s1', 'date25', 'gh_s1', 'mcs_s1', 'mh_s1', 'pcs_s1', 'pf_s1', 'genhth25', 'cmp1yr25', 'vigact25', 'modact25', 'lift25', 'climbs25', 'climb125', 'bend25', 'wk1ml25', 'wksblk25', 'wk1blk25', 'bathe25', 'phctdn25', 'phacls25', 'limit25', 'exefrt25', 'emctdn25', 'emacls25', 'carful25', 'probsa25', 'bdpain25', 'painin25', 'pep25', 'nrvous25', 'down25', 'calm25', 'energ25', 'blue25', 'worn25', 'happy25', 'tired25', 'hlthlm25', 'sickez25', 'hlthy25', 'worse25', 'exclnt25', 'rawbp_s1', 'rawgh_s1', 'rawmh_s1', 'rawpf_s1', 'rawre_s1', 'rawrp_s1', 'rawsf_s1', 'rawvt_s1', 're_s1', 'rp_s1', 'sf_s1', 'vt_s1']]

supplement2 = pd.read_csv(path + 'shhs2-dataset-0.15.0.csv', encoding='cp1252')
supplement2 = supplement2.rename(columns=str.lower)
supplement2 = supplement2[['nsrrid', 'age_s2', 'gender', 'race','bp_s2', 'formdate_ql', 'gh_s2', 'mcs_s2', 'mh_s2', 'pcs_s2', 'pf_s2', 'ql201', 'ql202', 'ql203a', 'ql203b', 'ql203c', 'ql203d', 'ql203e', 'ql203f', 'ql203g', 'ql203h', 'ql203i', 'ql203j', 'ql204a', 'ql204b', 'ql204c', 'ql204d', 'ql205a', 'ql205b', 'ql205c', 'ql206', 'ql207', 'ql208', 'ql209a', 'ql209b', 'ql209c', 'ql209d', 'ql209e', 'ql209f', 'ql209g', 'ql209h', 'ql209i', 'ql210', 'ql211a', 'ql211b', 'ql211c', 'ql211d', 'rawbp_s2', 'rawgh_s2', 'rawmh_s2', 'rawpf_s2', 'rawre_s2', 'rawrp_s2', 'rawsf_s2', 'rawvt_s2', 're_s2', 'rp_s2', 'sf_s2', 'vt_s2']]

sleep_sess_1 = sleep_pivot.loc[sleep_pivot['session'] == 1]
sleep_sess_2 = sleep_pivot.loc[sleep_pivot['session'] == 2]

sleep_sess_1 = sleep_sess_1.merge(supplement1, how='inner', on = 'nsrrid')
sleep_sess_2 = sleep_sess_2.merge(supplement2, how='inner', on = 'nsrrid')

sleep_sess_1 = sleep_sess_1.rename(columns={'bp_s1': 'bp_s2', 'date25': 'formdate_ql', 'gh_s1': 'gh_s2', 'mcs_s1': 'mcs_s2', 'mh_s1': 'mh_s2', 'pcs_s1': 'pcs_s2', 'pf_s1': 'pf_s2', 'genhth25': 'ql201', 'cmp1yr25': 'ql202', 'vigact25': 'ql203a', 'modact25': 'ql203b', 'lift25': 'ql203c', 'climbs25': 'ql203d', 'climb125': 'ql203e', 'bend25': 'ql203f', 'wk1ml25': 'ql203g', 'wksblk25': 'ql203h', 'wk1blk25': 'ql203i', 'bathe25': 'ql203j', 'phctdn25': 'ql204a', 'phacls25': 'ql204b', 'limit25': 'ql204c', 'exefrt25': 'ql204d', 'emctdn25': 'ql205a', 'emacls25': 'ql205b', 'carful25': 'ql205c', 'probsa25': 'ql206', 'bdpain25': 'ql207', 'painin25': 'ql208', 'pep25': 'ql209a', 'nrvous25': 'ql209b', 'down25': 'ql209c', 'calm25': 'ql209d', 'energ25': 'ql209e', 'blue25': 'ql209f', 'worn25': 'ql209g', 'happy25': 'ql209h', 'tired25': 'ql209i', 'hlthlm25': 'ql210', 'sickez25': 'ql211a', 'hlthy25': 'ql211b', 'worse25': 'ql211c', 'exclnt25': 'ql211d', 'rawbp_s1': 'rawbp_s2', 'rawgh_s1': 'rawgh_s2', 'rawmh_s1': 'rawmh_s2', 'rawpf_s1': 'rawpf_s2', 'rawre_s1': 'rawre_s2', 'rawrp_s1': 'rawrp_s2', 'rawsf_s1': 'rawsf_s2', 'rawvt_s1': 'rawvt_s2', 're_s1': 're_s2', 'rp_s1': 'rp_s2', 'sf_s1': 'sf_s2', 'vt_s1': 'vt_s2'})

final_sess = pd.concat([sleep_sess_1, sleep_sess_2])
final_sess.to_csv(path + 'aggregated_sleep_data.csv')

In [11]:
final_sess

,nsrrid,session,0,1,2,3,5,total_stage,w%,n1%,...,rawpf_s2,rawre_s2,rawrp_s2,rawsf_s2,rawvt_s2,re_s2,rp_s2,sf_s2,vt_s2,age_s2
0,200079,1,21.0,14.0,526.0,35.0,34.0,630.0,3.333333,2.222222,...,20.000000,6.0,4.0,6.0,5.0,100.000000,0.0,50.0,5.0,NaN
1,200082,1,52.0,33.0,462.0,151.0,182.0,880.0,5.909091,3.750000,...,30.000000,6.0,8.0,10.0,15.0,100.000000,100.0,100.0,55.0,NaN
2,200108,1,86.0,24.0,432.0,113.0,155.0,810.0,10.617284,2.962963,...,27.000000,4.0,7.0,9.0,14.0,33.333333,75.0,87.5,50.0,NaN
3,200109,1,85.0,62.0,301.0,336.0,86.0,870.0,9.770115,7.126437,...,28.000000,6.0,8.0,10.0,20.0,100.000000,100.0,100.0,80.0,NaN
4,200112,1,82.0,39.0,472.0,73.0,144.0,810.0,10.123457,4.814815,...,30.000000,6.0,8.0,10.0,16.0,100.000000,100.0,100.0,60.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,205654,2,200.0,52.0,573.0,15.0,160.0,1000.0,20.000000,5.200000,...,NaN,NaN,NaN,10.0,15.0,NaN,NaN,100.0,55.0,72.0
491,205655,2,33.0,11.0,341.0,188.0,187.0,760.0,4.342105,1.447368,...,27.777778,6.0,8.0,10.0,16.0,100.000000,100.0,100.0,60.0,63.0
492,205657,2,217.0,41.0,399.0,71.0,112.0,840.0,25.833333,4.880952,...,28.888889,6.0,8.0,10.0,16.0,100.000000,100.0,100.0,60.0,68.0
493,205668,2,94.0,85.0,591.0,97.0,233.0,1100.0,8.545455,7.727273,...,22.222222,6.0,7.0,10.0,18.0,100.000000,75.0,100.0,70.0,73.0
